In [ ]:
# load packages
## some of them might not be used in this file

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV

from sklearn.tree import plot_tree

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split

In [ ]:
# Evaluation on training set
from sklearn.metrics import r2_score
def R2(model, X_train,y_train):
    
    y_pred = model.predict(X_train)
    SSE = np.sum((y_train - y_pred)**2)
    SST = np.sum((y_train - np.mean(y_train))**2)
                 
    return (1 - SSE/SST)

# Evaluation on test set
def OSR2(model, X_test, y_test, y_train):
    
    y_pred = model.predict(X_test)
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(y_train))**2)
                 
    return (1 - SSE/SST)

In [ ]:
# scale columns
def z_scale(df,y_name):
    zscore=StandardScaler().fit(np.array(df[y_name]).reshape(-1, 1)) # 按原始训练集生成规则，即训练的均值和标准差
    test=zscore.transform(np.array(df[y_name]).reshape(-1, 1))
    df[y_name+'_scale']=test 
    df[y_name+'_scale']=round(df[y_name+'_scale'],5)#保留五位小数

In [ ]:
##one std rule

def one_standard_error_rule(model, results, param_grid, n_splits):
    
    #assert neg_mean_squared_error == True # function is defined specifically for neg_mean_squared_error
    
    #find model with minimum error, then select the simplest model
    #whose mean falls within 1 standard deviation of the minimum
    
    range_x = param_grid # results['param_'+list(param_grid.keys())[0]].data
    std_vs_x  = pd.Series(results['std_test_score'], index = range_x)
    sem_vs_x  = std_vs_x/np.sqrt(n_splits)
    
    mean_vs_x = pd.Series(results['mean_test_score'], index = range_x)        
    mean_vs_x = mean_vs_x*(-1)
    
    x_min = mean_vs_x.idxmin()
    sem = sem_vs_x[x_min]
    
    x_1se = mean_vs_x[mean_vs_x <= min(mean_vs_x) + sem].index.min()
    

    #if (model=='pcr'):
        #x_1se = mean_vs_x[mean_vs_x <= min(mean_vs_x) + sem].index.min()
    #elif (model=='ridge') | (model=='lasso'):
        #x_1se = mean_vs_x[mean_vs_x <= min(mean_vs_x) + sem].index.max()
        
    #x_1se_idx = int(np.argwhere(range_x == x_1se)[0])
    
    return x_min, x_1se

In [ ]:
# Calculate Variance Inflation Factor for each explanatory variable

from statsmodels.stats.outliers_influence import variance_inflation_factor

def VIF(df, columns):
    
    values = sm.add_constant(df[columns]).values  # the dataframe passed to VIF must include the intercept term. We add it the same way we did before.
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    
    return pd.Series(vif[1:], index=columns)

In [ ]:
# load data
## some columns of the raw data won't be used in the final random forest model
pgv=pd.read_csv('pgv_ca_county.csv')
vul=pd.read_csv('vulnerability.csv')
socio_econ=pd.read_excel('socio_econ.xlsx')
resil=pd.read_excel('resilience.xlsx')
district=pd.read_csv('county_district.csv')

In [ ]:
# shrink dimension on pgv data
from sklearn.decomposition import PCA
pgv_2=pgv.copy().drop(['lon','lat','county','state'],axis=1)
pca = PCA(n_components=1)
pgv['pgv_pca'] = pca.fit_transform(pgv_2) 

# scale pgv_pca
z_scale(pgv,'pgv_pca')


In [ ]:
# standardize household income and total population in socio_econ

z_scale(socio_econ,'house_inc')
z_scale(socio_econ,'Total Population')


In [ ]:
# fill na with mean

resil['recovery'] = resil['recovery'].fillna(resil['recovery'].mean())
resil['resistance'] = resil['resistance'].fillna(resil['resistance'].mean())

In [ ]:
# merge dataset
raw_data=pd.merge(pgv,vul,left_on=['county'],right_on=['County'],how='left')
raw_data=pd.merge(raw_data,socio_econ,left_on=['county'],right_on=['NAME'],how='left')
raw_data=pd.merge(raw_data,resil,left_on=['county'],right_on=['County Name'],how='left')
raw_data=pd.merge(raw_data,socio_econ_reduced,left_on=['county'],right_on=['NAME'],how='left')
raw_data=pd.merge(raw_data,district,left_on=['county'],right_on=['County'],how='left')



#raw['house_inc']=raw['house_inc'].astype('float')

#earthquake risk index
#raw_data['earthquake_risk']=raw_data.apply(lambda row:row['pgv_pca_scale']*row['vulnerability_index'],axis=1)



In [ ]:
raw_data.columns

#### Final Random Forest Model: Using `pgv_pca` and `vulnerability index` seperately with `resistance`

In [ ]:
# useful subset of raw data
df_ls=['lon', 'lat','county','District','pgv_pca_scale','vulnerability_index','pop_over65'
       , 'disab_pct', 'edu_attain', 'unemployment',
       'inc_inequal', 'health_insure_lack','mobile_home_pct',
       'house_owner_pct', 'vacant_rent_pct', 'house_wo_vehi_pct', 'connect',
       'hosp_cap', 'medi_prof_cap', 'school_cap', 'popul_change',
       'hot_mot_cap', 'house_inc_scale', 'Total Population_scale','resistance']

df_raw=raw_data[df_ls]
df_raw.head()

In [ ]:
# dataset for models
df=df_raw.drop(['lon','lat'],axis=1)
df.columns

In [ ]:
# split dataset 
df_train, df_test = train_test_split(df, test_size=0.3, random_state=11)

X_train=df_train.drop(['resistance'],axis=1)
y_train=df_train['resistance']

X_test=df_test.drop(['resistance'],axis=1)
y_test=df_test['resistance']

                     
df_train.shape,df_test.shape

In [ ]:
X_train_dtr=pd.get_dummies(X_train.drop(['District'],axis=1))

In [ ]:
len(X_train_dtr.columns)

#### 2. Overall RandomForest Regression Model

In [ ]:
#prepare data

X_train_dtr=pd.get_dummies(X_train.drop(['District'],axis=1))
X_test_dtr=pd.get_dummies(X_test.drop(['District'],axis=1))

#CV on max features

import time
grid_values = {'max_features': np.linspace(5,77,15, dtype='int32'),
              'n_estimators': [50],
              'random_state': [1010]} 

tic = time.time()

rf = RandomForestRegressor() 
rf_cv = GridSearchCV(rf, param_grid=grid_values, scoring='r2', cv=5)
rf_cv.fit(X_train_dtr, y_train)

toc = time.time()
print('time:', round(toc-tic, 2),'s')

In [ ]:
## plotting

max_features = rf_cv.cv_results_['param_max_features'].data
R2_scores = rf_cv.cv_results_['mean_test_score']

plt.figure(figsize=(8, 6))
plt.xlabel('max features', fontsize=16)
plt.ylabel('CV R2', fontsize=16)
plt.scatter(max_features, R2_scores, s=30)
plt.plot(max_features, R2_scores, linewidth=3)
plt.grid(True, which='both')
plt.xlim([5, 76])
plt.ylim([0.999, 1])
plt.title('CV on Max Features',fontsize=18)

In [ ]:
## one stand rule

from scipy import stats

n_components = rf_cv.cv_results_['param_max_features'].data
R2_scores = rf_cv.cv_results_['mean_test_score']
x_min, x_1se = one_standard_error_rule(model='rf_cv',
                                       results=rf_cv.cv_results_,
                                       param_grid=n_components,
                                       n_splits=15)

plt.figure(figsize=(8, 6))
plt.xlabel('max features', fontsize=16)
plt.ylabel('CV R2', fontsize=16)
plt.scatter(n_components, R2_scores, s=30)
plt.axvline(x=x_min, color='m')
plt.axvline(x=x_1se, color='c')
plt.grid(True, which='both')
plt.title('1std Rule',fontsize = 18)

plt.tight_layout()
plt.show()

In [ ]:
x_1se

In [ ]:
#prepare data

X_train_dtr=pd.get_dummies(X_train.drop(['District'],axis=1))
X_test_dtr=pd.get_dummies(X_test.drop(['District'],axis=1))

#Overall RandomForest Regression Model
rf1 = RandomForestRegressor(n_estimators = 50, random_state=1010,max_features=56)
rf1.fit(X_train_dtr, y_train)

# Feature Importance

feature_imp1 = pd.DataFrame({'Feature' : X_train_dtr.columns, 
              'Importance score': 100*rf1.feature_importances_}
                           ).round(1).sort_values(by='Importance score',ascending=False)
feature_imp1.head(40)



In [ ]:
print('Random Forest R2:', r2_score(y_train,rf1.predict(X_train_dtr)))
print('Random Forest OSR2:', round(OSR2(rf1, X_test_dtr, y_test, y_train), 10))


In [ ]:
# get important features for each district

def rfr_district(df_train,df_test,district):
    
    # prepare dataset
    df_train=df_train[df_train['District']==district]
    df_test=df_test[df_test['District']==district]
    
    X_train=df_train.drop(['District','resistance'],axis=1)
    y_train=df_train['resistance']
    X_test=df_test.drop(['District','resistance'],axis=1)
    y_test=df_test['resistance']
    
    # decision tree regressor
    ## cross validation on `max_features` of the overall model above doesn't provide a outstanding parameter,
    ## thus we decide to run the model with default value
    rfr = RandomForestRegressor(min_samples_leaf=10, min_samples_split=15,n_estimators = 50, random_state=1010)
    rfr.fit(X_train, y_train)
    
    # r2 and osr2
    #dtr_r2=r2_score(y_train,dtr.predict(X_train))
    r2=round(R2(rfr, X_train,y_train), 10)
    osr2=round(OSR2(rfr, X_test, y_test, y_train), 10)
    
    # Feature Importance
    feature_imp = pd.DataFrame({'Feature' : X_train.columns, 
              'Importance score': 100*rfr.feature_importances_}).round(10)
    feature_imp = feature_imp.sort_values(by='Importance score',ascending=False).reset_index(drop = True)
    
    return r2,osr2,feature_imp

In [ ]:
# create a table for top5 imprtant features of all districts
dist_ls=['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12']
Feature_1st_rfr=[]
ImptScore_1st_rfr=[]
Feature_2nd_rfr=[]
ImptScore_2nd_rfr=[]
Feature_3rd_rfr=[]
ImptScore_3rd_rfr=[]
Feature_4th_rfr=[]
ImptScore_4th_rfr=[]
Feature_5th_rfr=[]
ImptScore_5th_rfr=[]

for i in dist_ls:
    impt_df_rfr = rfr_district(df_train,df_test,i)[2]
    Feature_1st_rfr.append(impt_df_rfr['Feature'][0])
    ImptScore_1st_rfr.append(impt_df_rfr['Importance score'][0])
    Feature_2nd_rfr.append(impt_df_rfr['Feature'][1])
    ImptScore_2nd_rfr.append(impt_df_rfr['Importance score'][1])
    Feature_3rd_rfr.append(impt_df_rfr['Feature'][2])
    ImptScore_3rd_rfr.append(impt_df_rfr['Importance score'][2])
    Feature_4th_rfr.append(impt_df_rfr['Feature'][3])
    ImptScore_4th_rfr.append(impt_df_rfr['Importance score'][3])
    Feature_5th_rfr.append(impt_df_rfr['Feature'][4])
    ImptScore_5th_rfr.append(impt_df_rfr['Importance score'][4])
    
df_ls= [dist_ls,
        Feature_1st_rfr,ImptScore_1st_rfr,
        Feature_2nd_rfr,ImptScore_2nd_rfr,
        Feature_3rd_rfr,ImptScore_3rd_rfr, 
        Feature_4th_rfr,ImptScore_4th_rfr,
        Feature_5th_rfr,ImptScore_5th_rfr]

colname_ls=['District',
            'Feature_1st','ImptScore_1st',
            'Feature_2nd','ImptScore_2nd',
            'Feature_3rd','ImptScore_3rd',
            'Feature_4th','ImptScore_4th',
            'Feature_5th','ImptScore_5th']

rfr_result=pd.DataFrame(data=df_ls)

rfr_result = pd.DataFrame(rfr_result.values.T,columns=colname_ls)    
    

In [ ]:
rfr_result

In [ ]:
rfr_result.to_csv('rfr_result1.csv')